In [1]:
!pip install chromedriver_autoinstaller

In [ ]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import chromedriver_autoinstaller

MAX_OPTA_PAGE_NUM = 26  # Adjusted manually
OPTA_URL = 'https://dataviz.theanalyst.com/opta-power-rankings/'

def scrape_opta_club_rankings():
    chromedriver_autoinstaller.install()
    chrome_options = webdriver.ChromeOptions()
    options = ['--headless']
    
    for option in options:
        chrome_options.add_argument(option)

    driver = webdriver.Chrome(options=chrome_options)
    
    try:
        driver.get(OPTA_URL)
        time.sleep(3)
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        table = soup.find('table')
        if not table:
            print("No table found on the page.")
            return None

        headers = [th.text.strip() for th in table.find_all('th')]
        headers.append('id')

        rows = []
        page_num = 1

        while page_num <= MAX_OPTA_PAGE_NUM:
            print(f'Scraping page {page_num}')
            
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            table = soup.find('table')
            for tr in table.find_all('tr'):
                row = [td.text.strip() for td in tr.find_all('td')]
                img = tr.select_one('img')
                img_id = img['src'].split('&id=')[-1] if img else ''
                row.append(img_id)
                if row:
                    rows.append(row)
            
            if page_num < MAX_OPTA_PAGE_NUM:
                try:
                    buttons = driver.find_elements(by=By.CSS_SELECTOR, value='button')
                    last_button = buttons[-1]
                    last_button.click()
                except Exception as e:
                    print(f"Failed to click next on page {page_num}: {e}")
                    break
                
            time.sleep(2)
            page_num += 1

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        driver.quit()
        print('Done scraping Opta club rankings.')

    df = pd.DataFrame(rows, columns=headers)
    df.dropna(subset=['team'], inplace=True)  # Assuming 'team' is a column name
    return df

# Example usage:
df = scrape_opta_club_rankings()
if df is not None:
    print(df)


In [ ]:
df.to_csv('Opta Ratings No Leagues.csv', index=False)

In [9]:
import pandas as pd

# Define the file path
file_path = r'C:\Users\jbuck\Player-Recommendation-System\Player Metrics\Team Stats\Opta Ratings With Leagues.csv'

# Load the CSV file into a DataFrame
df = pd.read_csv(file_path)
import pandas as pd

# Assuming 'df' is your DataFrame and it's already defined somewhere in your environment

# Drop rows where the 'League' column has missing values
df.dropna(subset=['League'], inplace=True)

# Now 'df' will contain only rows where 'League' column has a non-null value
df



,rank,team,rating,ranking change 7 days,id,League
0,1,Manchester City,100.0,0,a3nyxabgsqlnqfkeg41m6tnpp,Premier League
1,2,Real Madrid,98.0,0,3kq9cckrnlogidldtdie2fkbl,La Liga
2,3,Internazionale,97.6,0,3vo5mpj7catp66nrwwqiuhuup,Serie A
3,4,PSG,95.5,0,2b3mar72yy8d6uvat1ka6tn3r,Ligue 1
4,5,Arsenal,95.0,2,4dsgumo7d4zupm2ugsvm4zm4d,Premier League
...,...,...,...,...,...,...
890,891,Excelsior,68.6,33,dqk062lu0vm8epvytbm6r4mmf,Eredivisie
909,910,AS Eupen,68.4,-40,9njsy633rh1tyri7l2jaju5o,Belgian Pro League
931,932,Vitesse,68.3,39,6hsriqr3ybvyg94w2k19oal50,Eredivisie
950,951,Rotherham United,68.2,-26,6m5s30n9ej0dqfk1hfyrlkhdw,Championship


In [10]:
df.tail(30)

,rank,team,rating,ranking change 7 days,id,League
464,465,Cardiff City,73.0,-59,3ash57c0q2tgok6gvzbsta93,Championship
467,468,Boavista,73.0,-23,3b4nxgvmwzstmoq6borytkwx1,Premiere Liga
480,481,Vasco da Gama,72.9,5,5ponlslulpugdlvd93n9yqu2b,Brazillian League
482,483,Plymouth Argyle,72.9,-30,5g1fnexeromjl0lvnxp8hpnhi,Championship
491,492,Heerenveen,72.7,-27,4vd2t5schmvvufrfib7f2vjdf,Eredivisie
513,514,Birmingham City,72.3,18,8kreqtde5jdktcib0kg7dknac,Championship
518,519,Estrela,72.3,-64,85nuf5511v5omdretfoh0c4k4,Premiere Liga
536,537,Huddersfield Town,72.1,-3,5n5baro467i8s8efkra3nhgux,Championship
554,555,RWDM,71.9,-60,4kzh86nfubfwsjzerl9pz73kl,Belgian Pro League
587,588,Deportivo Riestra,71.5,18,7gcwwr8m5docs6jqrn2stilie,Argentinian League


In [2]:
import pandas as pd
import subprocess

# Assuming RB_scores_df is defined elsewhere in your notebook
df.to_csv('Team Ratings.csv', index=False)

def run_command(command):
    result = subprocess.run(command, shell=True, text=True, capture_output=True)
    if result.stderr:
        print("Error:", result.stderr)
    else:
        print("Output:", result.stdout)

# Properly quote the filename to handle spaces
run_command('git add "Team Ratings.csv"')
run_command('git commit -m "Add or update Team Ratings.csv"')

# Make sure the branch 'main' exists by checking your GitHub repo or by running 'git branch' locally.
run_command('git push origin main')  # Adjust branch name if necessary


Output: 
Output: On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   ../Attacking Midfielders and Wingers/creating CAM and Winger scores.ipynb
	modified:   ../Center Backs/creating center back player role scores.ipynb
	modified:   ../Center Midfielders/creating CM scores.ipynb
	modified:   ../Forwards/creating Forward Scores.ipynb
	modified:   ../Left Backs/creating left back scores.ipynb
	modified:   ../Right Backs/creating right back scores.ipynb
	modified:   Creating Team Scores.ipynb
	modified:   Setting Up League Ratings.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	../.ipynb_checkpoints/
	../Attacking Midfielders and Wingers/.ipynb_checkpoints/
	../Center Backs/.ipynb_checkpoints/
	../Center Midfielders/.ipynb_checkpoints/
	../Forwards/.ipynb_checkpoint

In [3]:
import pandas as pd

# Assuming 'df' is already defined and loaded with your data

# Group by the 'League' column and calculate the mean rating and count of teams for each league
league_stats = df.groupby('League').agg(
    Average_Rating=('rating', 'mean'),  # Calculate the average rating
    Number_of_Teams=('rating', 'count')  # Count non-null ratings to get number of teams
).reset_index()

# Display the new DataFrame with average ratings and team counts per league
print(league_stats)


                League  Average_Rating  Number_of_Teams
0   Argentinian League       74.432143               28
1   Belgian Pro League       77.868750               16
2    Brazillian League       76.655000               20
3           Bundesliga       84.616667               18
4         Championship       75.733333               24
5           Eredivisie       74.755556               18
6              La Liga       84.840000               20
7              Ligue 1       84.050000               18
8       Premier League       86.810000               20
9        Premiere Liga       77.372222               18
10             Serie A       84.860000               20


In [4]:
# Define a dictionary for league name mapping
league_name_mapping = {
    'Argentinian League': 'ARG-League',
    'Belgian Pro League': 'BEL-Pro League',
    'Brazillian League': 'BRA-League',
    'Bundesliga': 'GER-Bundesliga',
    'Championship': 'ENG-Championship',
    'Eredivisie': 'NED-Eredivisie',
    'La Liga': 'ESP-La Liga',
    'Ligue 1': 'FRA-Ligue 1',
    'Premier League': 'ENG-Premier League',
    'Premiere Liga': 'POR-League',  # Assuming this matches to 'POR-League'
    'Serie A': 'ITA-Serie A'
}

# Apply the mapping to the 'League' column
league_stats['League'] = league_stats['League'].map(league_name_mapping)

# Display the updated DataFrame
print(league_stats)

                League  Average_Rating  Number_of_Teams
0           ARG-League       74.432143               28
1       BEL-Pro League       77.868750               16
2           BRA-League       76.655000               20
3       GER-Bundesliga       84.616667               18
4     ENG-Championship       75.733333               24
5       NED-Eredivisie       74.755556               18
6          ESP-La Liga       84.840000               20
7          FRA-Ligue 1       84.050000               18
8   ENG-Premier League       86.810000               20
9           POR-League       77.372222               18
10         ITA-Serie A       84.860000               20


In [5]:
# Sort the DataFrame by 'Average_Rating' in descending order
league_stats_sorted = league_stats.sort_values(by='Average_Rating', ascending=False)

# Round the 'Average_Rating' to two decimal points
league_stats_sorted['Average_Rating'] = league_stats_sorted['Average_Rating'].round(2)

# Display the sorted and rounded DataFrame
print(league_stats_sorted)

                League  Average_Rating  Number_of_Teams
8   ENG-Premier League           86.81               20
10         ITA-Serie A           84.86               20
6          ESP-La Liga           84.84               20
3       GER-Bundesliga           84.62               18
7          FRA-Ligue 1           84.05               18
1       BEL-Pro League           77.87               16
9           POR-League           77.37               18
2           BRA-League           76.66               20
4     ENG-Championship           75.73               24
5       NED-Eredivisie           74.76               18
0           ARG-League           74.43               28


In [6]:
import pandas as pd
import subprocess

# Assuming RB_scores_df is defined elsewhere in your notebook
league_stats_sorted.to_csv('League Ratings Cleaned.csv', index=False)

def run_command(command):
    result = subprocess.run(command, shell=True, text=True, capture_output=True)
    if result.stderr:
        print("Error:", result.stderr)
    else:
        print("Output:", result.stdout)

# Properly quote the filename to handle spaces
run_command('git add "League Ratings Cleaned.csv"')
run_command('git commit -m "Add or update League Ratings Cleaned.csv"')

# Make sure the branch 'main' exists by checking your GitHub repo or by running 'git branch' locally.
run_command('git push origin main')  # Adjust branch name if necessary


Output: 
Output: On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   ../Attacking Midfielders and Wingers/creating CAM and Winger scores.ipynb
	modified:   ../Center Backs/creating center back player role scores.ipynb
	modified:   ../Center Midfielders/creating CM scores.ipynb
	modified:   ../Forwards/creating Forward Scores.ipynb
	modified:   ../Left Backs/creating left back scores.ipynb
	modified:   ../Right Backs/creating right back scores.ipynb
	modified:   Creating Team Scores.ipynb
	modified:   Setting Up League Ratings.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	../.ipynb_checkpoints/
	../Attacking Midfielders and Wingers/.ipynb_checkpoints/
	../Center Backs/.ipynb_checkpoints/
	../Center Midfielders/.ipynb_checkpoints/
	../Forwards/.ipynb_checkpoint

In [7]:
import subprocess

def run_command(command):
    result = subprocess.run(command, shell=True, text=True, capture_output=True)
    if result.stderr:
        print("Error:", result.stderr)
    else:
        print("Output:", result.stdout)

# Add, commit, pull, and then push
run_command('git add "Setting Up League Ratings.ipynb"')
run_command('git commit -m "Update notebook with latest changes"')
run_command('git pull origin main')  # Pull before pushing to ensure the local repo is up-to-date
run_command('git push origin main')  # Adjust branch name if necessary


Error: warning: in the working copy of 'Player Metrics/Team Stats/Setting Up League Ratings.ipynb', LF will be replaced by CRLF the next time Git touches it

Output: [main caa76c8] Update notebook with latest changes
 1 file changed, 122 insertions(+), 75 deletions(-)

Error: From https://github.com/Jbuck1016/Player-Recommendation-System
 * branch            main       -> FETCH_HEAD

Error: To https://github.com/Jbuck1016/Player-Recommendation-System.git
   0507007..caa76c8  main -> main

